In [ ]:
import pandas as pd
import numpy as np
import random
import os
import joblib
import nltk
nltk.download('punkt')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.metrics import f1_score, classification_report, confusion_matrix

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm import tqdm

from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings("ignore")

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed()

def load_and_preprocess_data(file_path):
    data = pd.read_csv(file_path)
    data['symptoms'] = data['symptoms'].astype(str).str.lower()
    data['symptoms'] = data['symptoms'].str.replace(r'[^\w\s]', '', regex=True)
    label_encoder = LabelEncoder()
    data['Disease label'] = label_encoder.fit_transform(data['disease'])
    return data, label_encoder

def prepare_datasets(data, test_size=0.2):
    X = data['symptoms']
    y = data['Disease label']
    return train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

class BERTDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

class BERTDiagnoser:
    def __init__(self, model_name='roberta-base', max_len=128, batch_size=16, epochs=10):
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.max_len = max_len
        self.batch_size = batch_size
        self.epochs = epochs
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def encode_texts(self, texts):
        return self.tokenizer(list(texts), padding=True, truncation=True, max_length=self.max_len, return_tensors='pt')

    def create_data_loader(self, texts, labels):
        encodings = self.encode_texts(texts)
        dataset = BERTDataset(encodings, labels)
        return DataLoader(dataset, batch_size=self.batch_size)

    def train(self, train_loader, num_classes):
        model = AutoModelForSequenceClassification.from_pretrained(self.model_name, num_labels=num_classes).to(self.device)
        optimizer = AdamW(model.parameters(), lr=2e-5)
        model.train()
        for epoch in range(self.epochs):
            print(f'\n🧠 Epoch {epoch + 1}/{self.epochs}')
            for batch in tqdm(train_loader):
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                optimizer.zero_grad()
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                loss.backward()
                optimizer.step()
        return model

class TraditionalModels:
    def __init__(self):
        self.sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

    def vectorize_texts(self, texts):
        return self.sbert_model.encode(list(texts), show_progress_bar=False)

    def vectorize_text(self, text):
        return self.sbert_model.encode([text])[0]

    def train(self, X_train, y_train):
        X_train_vec = self.vectorize_texts(X_train)

        models = {
            'Logistic Regression': LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42),
            'SVM': SVC(kernel='linear', probability=True, class_weight='balanced', random_state=42),
            'XGBoost': xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42),
            'Random Forest': RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42),
            'Naive Bayes': GaussianNB()
        }

        for name, model in models.items():
            print(f"\n🚀 Training {name}...")
            model.fit(X_train_vec, y_train)
            preds = model.predict(X_train_vec)
            print(f"{name} F1-Score:\n{classification_report(y_train, preds)}")

        return models, self.sbert_model

    def predict_proba(self, models, sbert_model, text):
        vec = sbert_model.encode([text])[0].reshape(1, -1)
        results = {}
        for name, model in models.items():
            results[name] = model.predict_proba(vec)[0]
        return results

class DiseaseDiagnosisSystem:
    def __init__(self):
        self.models = {}
        self.label_encoder = None
        self.sbert_model = None
        self.bert_model = None
        self.bert_tokenizer = None
        self.model_dir = './saved_models'
        os.makedirs(self.model_dir, exist_ok=True)
        self.model_weights = {}

    def load_data(self, file_path):
        self.data, self.label_encoder = load_and_preprocess_data(file_path)
        self.X_train, self.X_test, self.y_train, self.y_test = prepare_datasets(self.data)

    def train_models(self):
        trad = TraditionalModels()
        self.models, self.sbert_model = trad.train(self.X_train, self.y_train)
        X_train_vec = trad.vectorize_texts(self.X_train)

        self.model_weights = {}
        for name, model in self.models.items():
            preds = model.predict(X_train_vec)
            f1 = f1_score(self.y_train, preds, average='weighted')
            self.model_weights[name] = f1

        bert_model = BERTDiagnoser()
        train_loader = bert_model.create_data_loader(self.X_train.values, self.y_train.values)
        self.bert_model = bert_model.train(train_loader, len(self.label_encoder.classes_))
        self.bert_tokenizer = bert_model.tokenizer

        preds = []
        with torch.no_grad():
            for text in self.X_train:
                inputs = self.bert_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(self.bert_model.device)
                outputs = self.bert_model(**inputs)
                pred = torch.argmax(outputs.logits, dim=1).cpu().item()
                preds.append(pred)
        f1 = f1_score(self.y_train, preds, average='weighted')
        self.model_weights['BERT'] = f1

      # ✅ Maintain order after training
        self.save_system()

    def save_system(self):
        try:
            joblib.dump((self.models, self.sbert_model), os.path.join(self.model_dir, 'traditional_models.pkl'))
            joblib.dump(self.label_encoder, os.path.join(self.model_dir, 'label_encoder.pkl'))
            self.bert_model.save_pretrained(os.path.join(self.model_dir, 'bert_model'))
            self.bert_tokenizer.save_pretrained(os.path.join(self.model_dir, 'bert_model'))
            joblib.dump(self.model_weights, os.path.join(self.model_dir, 'model_weights.pkl'))
            print("✅ The entire system has been saved successfully.")
        except Exception as e:
            print("❌ System save failed:", e)

    def load_system(self):
        try:
            self.models, self.sbert_model = joblib.load(os.path.join(self.model_dir, 'traditional_models.pkl'))
            self.label_encoder = joblib.load(os.path.join(self.model_dir, 'label_encoder.pkl'))
            self.bert_model = AutoModelForSequenceClassification.from_pretrained(os.path.join(self.model_dir, 'bert_model'))
            self.bert_tokenizer = AutoTokenizer.from_pretrained(os.path.join(self.model_dir, 'bert_model'))
            self.bert_model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
            self.model_weights = joblib.load(os.path.join(self.model_dir, 'model_weights.pkl'))
            print("✅ The full system has been successfully downloaded.")
            return True
        except Exception as e:
            print("❌ System loading failed:", e)
            return False

    def load_models(self):
        return self.load_system()

    def predict(self, symptoms):
        trad = TraditionalModels()
        trad_preds = trad.predict_proba(self.models, self.sbert_model, symptoms)
        inputs = self.bert_tokenizer(symptoms, return_tensors="pt", truncation=True, padding=True, max_length=128)
        inputs = {k: v.to(self.bert_model.device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = self.bert_model(**inputs)
            probs = torch.nn.functional.softmax(outputs.logits, dim=1).cpu().numpy()[0]
        trad_preds['BERT'] = probs

        weighted_probs = sum(self.model_weights[name] * trad_preds[name] for name in trad_preds)
        avg_probs = weighted_probs / sum(self.model_weights.values())

        pred_idx = np.argmax(avg_probs)
        predicted_disease = self.label_encoder.inverse_transform([pred_idx])[0]
        confidence = avg_probs[pred_idx]
        max_contributor = max(trad_preds.items(), key=lambda x: x[1][pred_idx])[0]
        top_diseases = [(self.label_encoder.inverse_transform([i])[0], avg_probs[i]) for i in np.argsort(avg_probs)[::-1]]

        return {
            'disease': predicted_disease,
            'confidence': confidence,
            'main_contributor': max_contributor,
            'details': trad_preds,
            'top_7': top_diseases[:7]
        }

    def evaluate_on_external_data(self, file_path):
        try:
            df = pd.read_csv(file_path)
            df['symptoms'] = df['symptoms'].astype(str).str.lower().str.replace(r'[^\w\s]', '', regex=True)
            true_labels = self.label_encoder.transform(df['disease'])

            pred_labels = []
            for i in range(len(df)):
                result = self.predict(df.iloc[i]['symptoms'])
                pred = self.label_encoder.transform([result['disease']])[0]
                pred_labels.append(pred)

            print("\n📊 Performance Evaluation:")
            print(classification_report(true_labels, pred_labels, target_names=self.label_encoder.classes_))
            print("\n🧩 Confusion Matrix:")
            print(confusion_matrix(true_labels, pred_labels))
        except Exception as e:
            print("❌ Failure to evaluate external data:", e)

    def interactive_diagnosis(self):
        print("\n🚀 Interactive Diagnostic System")
        print("Write down the symptoms one by one and then 'Diagnose' to diagnose, or 'Exit' to end the session.\n")
        symptoms_list = []

        while True:
            symptom = input("🔍 Enter a symptom or 'diagnose' or 'Exit': ").strip()
            if symptom.lower() in ['exit', 'quit','bay']:
                print("🛑 The system has been terminated.")
                break
            elif symptom.lower() in ['diagnose','ok']:
                if not symptoms_list:
                    print("⚠️ No symptoms entered.")
                    continue
                full_symptoms = " ".join(symptoms_list)
                result = self.predict(full_symptoms)
                print(f"\n✅ Diagnosis: {result['disease']} ({result['confidence']*100:.2f}%)")
                print(f"🏆 Best Contributing Model: {result['main_contributor']}")
                print("\n📊 Top 7 diseases:")
                for i, (disease, prob) in enumerate(result['top_7'], 1):
                    print(f"{i}. {disease}: {prob*100:.2f}%")
                symptoms_list = []
            elif symptom.lower() == 'retraining':
                self.load_data('/content/last_skin_disease_symptoms_dataset.csv')
                self.train_models()
                print("✅ Models have been retrained.")
            elif symptom.lower().startswith('evaluation'):
                parts = symptom.split()
                if len(parts) == 2:
                    self.evaluate_on_external_data(parts[1])
                else:
                    print("❌ Type: evaluate <file_path>")
            else:
                symptoms_list.append(symptom)

if __name__ == "__main__":
    system = DiseaseDiagnosisSystem()
    if not system.load_models():
        system.load_data('/content/last_skin_disease_symptoms_dataset.csv')
        system.train_models()
    system.interactive_diagnosis()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


✅ The full system has been successfully downloaded.

🚀 Interactive Diagnostic System
Write down the symptoms one by one and then 'Diagnose' to diagnose, or 'Exit' to end the session.

🔍 Enter a symptom or 'diagnose' or 'Exit': Exit
🛑 The system has been terminated.
